<a href="https://colab.research.google.com/github/binaryinferno/binaryinferno/blob/main/BinaryInferno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

将数据以十六进制形式提取出来，每条消息一行,您可以使用t-shark工具来完成此操作，但请注意要将数据从**封装的**TCP/UDP包中截取出来。
如果您知道系统是大端序还是小端序，请使用相应的标志运行该工具。这将限制该工具仅基于该端序构建描述。 

**detectors**

BE 用于大端序

LE 用于小端序

时间戳操作

如果消息长度都相同，则不会执行序列化模式搜索，因为我们假设字段长度是固定的。

序列化模式搜索是速度最慢的部分。我们通过一些技术上可行的最低成本的shell脚本来实现并行化。（更多的CPU有助于序列化模式搜索。我们在论文中使用了40个核心。并使用了128GB的内存）

在其中存在一个参数，用于设置特定偏移量搜索放弃的最长时间。

In [ ]:
# %%bash
# # Setup stuff

# # We need parallel because we use a shell script deep down to run the serialization pattern search in parallel
# apt -q install parallel > /dev/null 


# # We use this stuff to calculate entropy 
# pip3 install sklearn > /dev/null
# pip3 install scipy > /dev/null

# # Get a copy of the source.
# git clone https://github.com/binaryinferno/binaryinferno.git

Reading package lists...


Cloning into 'binaryinferno'...


创建一个输入文件叫input.txt，并填入字符，EOT是结束符，这是一种here-doc写法：
```
cat <<EOT > input.txt
这里是第一行的内容。
这里是第二行的内容。
EOT
```
## 案例1：

In [3]:
%%bash
cat <<EOT > input.txt
00000012000005d60004746573740a6b6b622d7562756e747500
0000001e000009f9030474657374175468697320697320612074657374206d65737361676521
00000017000007570304746573741048656c6c6f202d2074657374696e6721
000000150000068d021349276d20676f696e672061776179206e6f7721
EOT

In [5]:
%%bash

# log.txt包含BinaryInferno的详尽输出
# txt包含所有在stderr上出现的内容
# 我们主要关心log.txt最后的内容
(cd binaryinferno ; cat ../input.txt | python3 blackboard.py --detectors BE 1> ../log.txt 2> ../errs.txt )

# 查看log，匹配末尾的结果
cat log.txt | awk '/INFERRED DESCRIPTION/,/SPECEND/'

INFERRED DESCRIPTION
--------------------------------------------------------------------------------

	LLLLLLLL | ?????????? RRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRR
	--
	00000012 | 000005D600 04746573740A6B6B622D7562756E747500
	0000001E | 000009F903 0474657374175468697320697320612074657374206D65737361676521
	00000017 | 0000075703 04746573741048656C6C6F202D2074657374696E6721
	00000015 | 0000068D02 1349276D20676F696E672061776179206E6F7721
	--
	0 L BE UINT32 LENGTH + 8 = TOTAL MESSAGE LENGTH 16.0
	1 ? UNKNOWN TYPE 5 BYTE(S) 20.0
	2 R 0T_1L_V_BIG* 88.0

QTY SAMPLES
4
HEADER ONLY
LLLLLLLL | ?????????? RRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRR
SPECSTART
Length 4V_BE (BE uint32 Length + 8 = Total Message Length)
FieldFixed 5V (Unknown Type 5 Byte(s))
FieldRep *Q_0T_1L_1V_BE (0T_1L_V_big*)
SPECEND


上面的代码应该产生如下的输出

```
INFERRED DESCRIPTION
--------------------------------------------------------------------------------

	LLLLLLLL | ?????????? RRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRR
	--
	00000012 | 000005D600 04746573740A6B6B622D7562756E747500
	0000001E | 000009F903 0474657374175468697320697320612074657374206D65737361676521
	00000017 | 0000075703 04746573741048656C6C6F202D2074657374696E6721
	00000015 | 0000068D02 1349276D20676F696E672061776179206E6F7721
	--
	0 L BE UINT32 LENGTH + 8 = TOTAL MESSAGE LENGTH 16.0
	1 ? UNKNOWN TYPE 5 BYTE(S) 20.0
	2 R 0T_1L_V_BIG* 88.0

QTY SAMPLES
4
HEADER ONLY
LLLLLLLL | ?????????? RRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRR
SPECSTART
Length 4V_BE (BE uint32 Length + 8 = Total Message Length)
FieldFixed 5V (Unknown Type 5 Byte(s))
FieldRep *Q_0T_1L_1V_BE (0T_1L_V_big*)
SPECEND
```



## 案例2：

In [10]:
%%bash

# This is the example we show in the paper 
# Setup our input file with our hex messages (one message per line)
cat <<EOT > input.txt
01000D60A67AED054150504C45
01001160A67B0504504C554D0450454152
01000E60A67AF9064F52414E4745
EOT

# Tslow是时间戳的下界
# Tshigh是时间戳的上界
# Don't worry if it's years too wide, that's totally fine
(cd binaryinferno; cat ../input.txt | python3 blackboard.py --detectors BE --tslow "2001-02-08 11:41:41" --tshigh "'2028-02-08 11:41:41'" 1> ../log.txt 2> ../errs.txt )

cat log.txt | awk '/INFERRED DESCRIPTION/,/SPECEND/'

INFERRED DESCRIPTION
--------------------------------------------------------------------------------

	?? LLLL | TTTTTTTT RRRRRRRRRRRR
	--
	01 000D | 60A67AED 054150504C45
	01 0011 | 60A67B05 04504C554D0450454152
	01 000E | 60A67AF9 064F52414E4745
	--
	0 ? UNKNOWN TYPE 1 BYTE(S) 3.0
	1 L BE UINT16 LENGTH + 0 = TOTAL MESSAGE LENGTH 6.0
	2 T BE 32BIT SPAN SECONDS 2001-02-08 11:41:41.000000 TO 2028-02-08 11:41:41.000000 1.0 12.0
	3 R 0T_1L_V_BIG* 23.0

QTY SAMPLES
3
HEADER ONLY
?? LLLL | TTTTTTTT RRRRRRRRRRRR
SPECSTART
FieldFixed 1V (Unknown Type 1 Byte(s))
Length 2V_BE (BE uint16 Length + 0 = Total Message Length)
FieldFixed 4V_BE (BE 32BIT SPAN Seconds 2001-02-08 11:41:41.000000 to 2028-02-08 11:41:41.000000 1.0)
FieldRep *Q_0T_1L_1V_BE (0T_1L_V_big*)
SPECEND


结果分析：
```
推理结果描述：
--------------------------------------------------------------------------------
协议格式和类型描述：
	?? LLLL | TTTTTTTT RRRRRRRRRRRR
	-- 采样点
	01 000D | 60A67AED 054150504C45
	01 0011 | 60A67B05 04504C554D0450454152
	01 000E | 60A67AF9 064F52414E4745
	--
    每个字段的描述：序号、类型、大小端、数据类型、描述
	0 ? UNKNOWN TYPE 1 BYTE(S) 3.0
	1 L BE UINT16 LENGTH + 0 = TOTAL MESSAGE LENGTH 6.0
	2 T BE 32BIT SPAN SECONDS 2001-02-08 11:41:41.000000 TO 2028-02-08 11:41:41.000000 1.0 12.0
	3 R 0T_1L_V_BIG* 23.0

QTY SAMPLES
3
HEADER ONLY
?? LLLL | TTTTTTTT RRRRRRRRRRRR
SPECSTART
FieldFixed 1V (Unknown Type 1 Byte(s))
Length 2V_BE (BE uint16 Length + 0 = Total Message Length)
FieldFixed 4V_BE (BE 32BIT SPAN Seconds 2001-02-08 11:41:41.000000 to 2028-02-08 11:41:41.000000 1.0)
FieldRep *Q_0T_1L_1V_BE (0T_1L_V_big*)
SPECEND
```

In [14]:
%%bash
cd binaryinferno ;python3 blackboard.py --help

usage: blackboard.py [-h] [--filename FILENAME]
                     [--detectors DETECTORS [DETECTORS ...]] [--sigmaonly]
                     [--recurse] [--n MAXMSGS] [--tslow TSLOW]
                     [--tshigh TSHIGH]

Process some integers.

optional arguments:
  -h, --help            show this help message and exit
  --filename FILENAME   Filename to Read Input From
  --detectors DETECTORS [DETECTORS ...]
                        use only a single specific detector
  --sigmaonly           Report Only the Final Inferred description
  --recurse             Attempt Recursive Inference on Unknown portions
  --n MAXMSGS           Maximum Number of Messages to Use for Inference
  --tslow TSLOW         Datetime Range Low
  --tshigh TSHIGH       Datetime Range High
